# Probabilités & Statistiques  
  
### Septembre 2022
<br/>

# Série 01

---

In [1]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)
PDF('res/serie01.pdf', size=(800, 400))

## 1. Dénombrabilité
---
### (a)
On cherche à écrire un programme qui donné un `n` nous donne le n-ème élément de $A^{*} = {\{0,1\}^{*}}$

A* est ordonné selon l'ordre lexicographique, i.e. $A^*=\{0; 1; 00; 01; 10; 11; 000; 001; 010 : 011; 100; 101; 110; 111;. . .\}$  
<br/>

Définisson ${B=A^{*}}\ $ par simplicité. Si on sépare $B$ en "groupe" d'éléments de même tailles (i.e. les 0,1, les 00, 01 ...),  
on se rend compte que le nombre d'élément de chaque groupe est donné par $2^k$ où $k$ est la taille des éléments de ce dernier.  
On a donc que le nombre d'éléments (possibilités de combinaison de 0,1) de taille $\leq k$ est
$\sum^{k}_{i=1}{2^i} = 2^{i+1}-2$  
<br/>

Avec notre `n` donné, on cherche quelle taille ($k$) correspond à au moins `n` éléments:  
$2^{k+1}-2 = n \Longleftrightarrow 2^{k} = \frac{n}{2}+1 \Longleftrightarrow k = \log_2{( \frac{n}{2} + 1 )}$  
On obtient donc la taille du n-ème élément de $B$ comme $|B_n| = \lceil{\log_2{( \frac{n}{2} + 1 )}}\rceil$

In [2]:
import math
def lg(n):
    """return length of n-th element of B"""
    return math.ceil(math.log(n/2 + 1, 2))
def sum_group_card(n):
    return 2**(n+1) - 2

def getIndexInGroup(n):
    """Return the index of B_n among all elements of same length i.e. 'it is the k-th element of length lg(n)''"""
    length = lg(n)
    indexFromGroupStart = n - sum_group_card(length-1)
    return indexFromGroupStart

#### Tests début:

In [3]:
n = 13
print(f"size of {n}-th element: {lg(n)}")
print("index in group:", getIndexInGroup(n))

size of 13-th element: 3
index in group: 7


Maintenant qu'on a la position de $B_n$ parmis les éléments de la même taille que ce dernier, il suffit juste de convertir ce nombre en binaire et rajouter des 0 (padd) devant jusqu'à atteindre la taille désirée.

In [4]:
def padd(s, newLen):
    """Padd string with '0' until given length"""
    crtLen = len(s)
    if crtLen >= newLen: return s
    paddNb = newLen - crtLen
    return paddNb*"0" + s

def toBinPadd(n, newLen):
    """Convert to binary removes the '0b' prefix and padd with 0s"""
    return padd(str(bin(n))[2:], newLen)

In [5]:
def produce_nth_element(n):
    length = lg(n)
    idxGrp = getIndexInGroup(n) - 1 # Start to count at 1, therefore need to substract 1 from binary number
    return toBinPadd(idxGrp, length)

#
# Test de la fonction finale :
for i in range(1, 20):
    print(f"i={i}: \"{produce_nth_element(i)}\"")

i=1: "0"
i=2: "1"
i=3: "00"
i=4: "01"
i=5: "10"
i=6: "11"
i=7: "000"
i=8: "001"
i=9: "010"
i=10: "011"
i=11: "100"
i=12: "101"
i=13: "110"
i=14: "111"
i=15: "0000"
i=16: "0001"
i=17: "0010"
i=18: "0011"
i=19: "0100"



<br/>

---

<br/>

### (b)  
On sait qu'il existe une bijection de $\mathbb{N}$ vers les entiers naturels pair (que l'on notera $2\mathbb{N}$).  De la même manière, il en existe une entre $\mathbb{N}$ et $2\mathbb{N}+1$ (les nombres impairs). On obtient donc :
$ \begin{equation*}
    \begin{cases}
        x(n) := n \mapsto 2n & \\
        y(n) := n \mapsto 2n+1 &
    \end{cases}
\end{equation*} \ \ $ 

c'est à dire: 
$$
\begin{align*}
b \ : \ \mathbb{N} & \rightarrow \mathbb{N} \times \mathbb{N} \\
 n & \mapsto (2n, \ 2n+1)
\end{align*}
$$
Voir l'implémentation ci-dessous.

In [6]:
def b(n):
    def x(n): return 2*n
    def y(n): return 2*n + 1
    return (x(n), y(n))

(Pour se convaincre que les bijections existent vraiment, prenons le cas de $\mathbb{N} \rightarrow 2\mathbb{N}$.  
On sait que la fonction $x\mapsto 2x\ $ est injective ($2x=2y \leftrightarrow x=y)\ $ 
donc son image $F$ est au moins "aussi grande" que son ensemble de définition ce qui montre que $F$ est non-fini. Or on sait que $F \subset \mathbb{Z}$ et $\mathbb{Z}$ est dénombrable donc $F$ est aussi dénombrable et donc en bijection avec $\mathbb{N}$.)

In [7]:
for i in range(0, 15):
    print(b(i))

(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)
(10, 11)
(12, 13)
(14, 15)
(16, 17)
(18, 19)
(20, 21)
(22, 23)
(24, 25)
(26, 27)
(28, 29)


<br/>

---
<br/>

### (c)

Insérer diagonalisation de Cantor


 <br/>
 
## 2. Théorie des ensembles et mesures de probabilité
---

### (a)

On veut exprimer $\mathbb{1}_{A\cup B}\ $ en fonction de  $\mathbb{1}_A$ et  $\mathbb{1}_B\ $.
Considérons $(\mathbb{1}_{\Omega} - \mathbb{1}_A)(\mathbb{1}_{\Omega} - \mathbb{1}_{B})\ \ $, on a que 
$\mathbb{1}_{A \cup B}(x) = 0\ $ si et seulement si $x \notin A \wedge x \notin B\ \ \ $
i.e. Ssi $\mathbb{1}_A = 0 \wedge \mathbb{1}_{B} = 0\ \ $, ce qui donnerait $(\mathbb{1}_{\Omega} - 0)(\mathbb{1}_{\Omega} - 0) = \mathbb{1}_{\Omega}$.  
<br/>  

Si $x \in A \vee x \in B\ \ $ alors $\mathbb{1}_{\Omega} = 1 \wedge (\mathbb{1}_{\Omega} - \mathbb{1}_A) = 0\ \ $ ou $(\mathbb{1}_{\Omega} -\mathbb{1}_{B}) = 0\ \ $. On s'apperçoit que $(\mathbb{1}_{\Omega}-\mathbb{1}_A)(\mathbb{1}_{\Omega}-\mathbb{1}_{B})\ \ $ vaut toujours 0 sauf si $\mathbb{1}_{A \cup B}\ $ vaut 0. On peut donc juste rajouter $"\mathbb{1}_{\Omega} \ -"$ devant pour réquilibrer le tout et on finit avec $\mathbb{1}_{A \cup B}= \mathbb{1}_{\Omega} - (\mathbb{1}_{\Omega} - \mathbb{1}_A)(\mathbb{1}_{\Omega} -\mathbb{1}_{B})$.
